<a href="https://colab.research.google.com/github/augustompm/course-multiobjective-optimization/blob/main/multiobjetive_optimization_c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-a8nkvajw
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-a8nkvajw
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 801584cceb559adc54e828ebe9b385c5f53fe70f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10743 sha256=3a570d3ab74e9051a615aa78d54722c78a69a0c83a1d407fccde14c37a30e96c
  Stored in directory: /tmp/pip-ephem-wheel-cache-wozgycp_/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [3]:
%load_ext nvcc4jupyter



Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpwrke9eab".


In [ ]:
%%cuda
//Olá mundo CUDA: imprimindo de 1 até 10
#include <iostream>
int main() {
    for (int i = 1; i <= 10; ++i) {
        std::cout << i << " ";
    }
    std::cout << std::endl;
    return 0;
}


1 2 3 4 5 6 7 8 9 10 



In [14]:
%%cuda

//Trabalho 1: Problema da Mochila, Heurística Construtiva Aleatória, média 13.8 em 30 execuções

#include <iostream>
#include <vector>
#include <cstdlib>
#include <ctime>

using namespace std;

struct Item {
    int peso;
    int valor;
};

vector<bool> heuristicaAleatoria(const vector<Item>& itens, int capacidade, int tempoLimite) {
    vector<bool> solucao(itens.size(), false);
    int pesoAtual = 0;
    time_t inicio = time(0);

    while (difftime(time(0), inicio) < tempoLimite) {
        int indice = rand() % itens.size();
        if (!solucao[indice] && pesoAtual + itens[indice].peso <= capacidade) {
            solucao[indice] = true;
            pesoAtual += itens[indice].peso;
        }
    }

    return solucao;
}

int calcularFO(const vector<Item>& itens, const vector<bool>& solucao) {
    int valorTotal = 0;
    for (int i = 0; i < itens.size(); ++i) {
        if (solucao[i]) {
            valorTotal += itens[i].valor;
        }
    }
    return valorTotal;
}

double calcularMedia(const vector<int>& resultados) {
    int soma = 0;
    for (int i = 0; i < resultados.size(); ++i) {
        soma += resultados[i];
    }
    return static_cast<double>(soma) / resultados.size();
}

int main() {
    srand(time(0));

    int capacidade = 10;
    vector<Item> itens = {
        {1, 1},
        {1, 2},
        {1, 3},
        {5, 7},
        {5, 8}
    };

    int tempoLimite = 2;
    vector<int> resultados;

    for (int i = 0; i < 30; ++i) {
        vector<bool> solucao = heuristicaAleatoria(itens, capacidade, tempoLimite);
        int fo = calcularFO(itens, solucao);
        resultados.push_back(fo);
    }

    cout << "Resultados das execuções:\n";
    for (int i = 0; i < resultados.size(); ++i) {
        cout << resultados[i] << " ";
    }
    cout << endl;

    double media = calcularMedia(resultados);
    cout << "Média dos resultados: " << media << endl;

    return 0;
}


Resultados das execuções:
13 14 15 13 15 13 13 14 14 14 14 15 13 15 14 13 13 13 15 14 14 14 13 14 15 14 13 14 13 13 
Média dos resultados: 13.8



In [13]:
%%cuda

// Trabalho 1: Problema da Mochila, Heurística Construtiva Gulosa - média 14, superior

#include <iostream>
#include <vector>
#include <algorithm>

using namespace std;

// armazenar os dados dos itens
struct Item {
    int peso;
    int valor;
    int indice;  // Para rastrear o índice original

// Função para calcular a razão valor/peso
    double valorPorPeso() const {
        return (double) valor / peso;
    }
};

// solução gulosa baseada no valor/peso de cada item
vector<bool> heuristicaGulosa(const vector<Item>& itens, int capacidade) {
    vector<bool> solucao(itens.size(), false); // Solução inicial (vazia)
    int pesoAtual = 0; // Peso total na mochila

    // ordena os itens com base na razão valor/peso em ordem decrescente
    vector<Item> itensOrdenados = itens;
    sort(itensOrdenados.begin(), itensOrdenados.end(), [](const Item& a, const Item& b) {
        return a.valorPorPeso() > b.valorPorPeso(); // Ordena por valor/peso
    });

    // adiciona os itens até que a mochila atinja a capacidade
    for (const auto& item : itensOrdenados) {
        if (pesoAtual + item.peso <= capacidade) {
            solucao[item.indice] = true; // Adiciona o item à solução
            pesoAtual += item.peso;      // Atualiza o peso total
            // cout << "Item " << item.indice + 1 << " adicionado. Peso: " << item.peso << ", Valor: " << item.valor << endl;
        }
    }

    return solucao;
}

// função para calcular o valor total (FO)
int calcularFO(const vector<Item>& itens, const vector<bool>& solucao) {
    int valorTotal = 0;
    for (int i = 0; i < itens.size(); ++i) {
        if (solucao[i]) {
            valorTotal += itens[i].valor;
        }
    }
    return valorTotal;
}

void imprimirSolucao(const vector<Item>& itens, const vector<bool>& solucao) {
    cout << "\nItens:\n";
    for (int i = 0; i < itens.size(); ++i) {
        if (solucao[i]) {
            cout << "Item " << i + 1 << " [P: " << itens[i].peso << ", V: " << itens[i].valor << "]\n";
        }
    }
    cout << "\nFunção Objetivo: " << calcularFO(itens, solucao) << endl;
}

int main() {

    int capacidade = 10;
    vector<Item> itens = {
        {1, 1, 0},  // Item 1: Peso 1, Valor 1
        {1, 2, 1},  // Item 2: Peso 1, Valor 2
        {1, 3, 2},  // Item 3: Peso 1, Valor 3
        {5, 7, 3},  // Item 4: Peso 5, Valor 7
        {5, 8, 4}   // Item 5: Peso 5, Valor 8
    };

    vector<bool> solucao = heuristicaGulosa(itens, capacidade);

    imprimirSolucao(itens, solucao);

    return 0;
}



Itens:
Item 1 [P: 1, V: 1]
Item 2 [P: 1, V: 2]
Item 3 [P: 1, V: 3]
Item 5 [P: 5, V: 8]

Função Objetivo: 14

